In [ ]:
!pip install gradio

In [ ]:
pip install --upgrade gradio


In [83]:
from functools import partial


# loading missing value dataset

df_missing = pd.read_csv("Doctors_by_countries.csv") 

# loading cleaned datatset
dataset_file = "df_cleaned.csv"
df = pd.read_csv(dataset_file)
df.sort_values(by=['Countries, territories and areas', 'Year'], inplace=True)




In [87]:
import matplotlib.pyplot as plt
import io
from PIL import Image
import seaborn as sns


def filter_dataset(countries, year_min, year_max, doctor_min):
    filtered = df.copy()

    # filtering by list of countries
    if "All" not in countries and len(countries) > 0:
        filtered = filtered[filtered["Countries, territories and areas"].isin(countries)]

    # filtering by Min and Max Year
    filtered = filtered[(filtered["Year"] >= year_min) & (filtered["Year"] <= year_max)]
    
    # Doctor filter
    filtered = filtered[filtered["Medical doctors (number)"] >= doctor_min]
    
    return filtered, filtered 


def summary(filtered):
    desc = filtered.describe()

    total_doctors = filtered["Medical doctors (number)"].sum()
    avg_doctors_per_10k = filtered["Medical doctors (per 10 000 population)"].mean()
    num_countries = filtered["Countries, territories and areas"].nunique()
    year_min = filtered["Year"].min()
    year_max = filtered["Year"].max()

    kpi_text = (
        f"**Total doctors:** {total_doctors:,.0f}\n\n"
        f"**Average doctors per 10,000 population:** {avg_doctors_per_10k:.2f}\n\n"
        f"**Number of countries:** {num_countries}\n\n"
        f"**Year range:** {year_min} - {year_max}"
    )

    generalist = filtered["Generalist medical practitioners (number)"].sum()
    specialist = filtered["Specialist medical practitioners (number)"].sum()
    not_defined = filtered["Medical doctors not further defined (number)"].sum()

    labels = ["Generalist", "Specialist", "Not further defined"]
    sizes = [generalist, specialist, not_defined]
    colors = ["#66b3ff", "#99ff99", "#ffcc99"]

    plt.figure(figsize=(4, 4))
    plt.pie(sizes, labels=labels, colors=colors, autopct="%1.1f%%", startangle=140)
    plt.title("Doctor Types Proportion")
    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)
    pil_img = Image.open(buf) 

    return desc, kpi_text, pil_img

def fig_to_pil(fig):
    import io
    from PIL import Image
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    img = Image.open(buf)
    plt.close(fig)
    return img
    

def visualization(visualization_types, filtered):
    # filtered = df.copy()
    images = []
    
    x_col = "Year"
    y_col = "Medical doctors (number)"
    group_col = "Countries, territories and areas"
    
    for vis_type in visualization_types:
        fig, ax = plt.subplots(figsize=(6, 4))
        
        if vis_type == "Heatmap":
            # Example: correlation heatmap
            sns.heatmap(filtered.corr(numeric_only=True), annot=True, cmap="coolwarm", ax=ax)
            ax.set_title("Correlation Heatmap")
        
        elif vis_type == "Bar Graph":
            # Example: mean doctors per country
            filtered.groupby(group_col)[y_col].mean().sort_values().plot.bar(ax=ax)
            ax.set_title("Average Doctors per Country")
            ax.set_ylabel("Doctors")
        
        elif vis_type == "Line Graph":
            # Example: doctors over years (sum)
            filtered.groupby(x_col)[y_col].sum().plot(ax=ax)
            ax.set_title("Doctors Over Years")
            ax.set_ylabel("Doctors")
        
        elif vis_type == "Box Plot":
            # Example: doctors distribution per country
            sns.boxplot(x=group_col, y=y_col, data=filtered, ax=ax)
            ax.set_title("Doctors Distribution by Country")
            ax.set_ylabel("Doctors")
            plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
        
        elif vis_type == "Scatter Plot":
            # Example: scatter plot of year vs doctors
            sns.scatterplot(x=x_col, y=y_col, data=filtered, ax=ax)
            ax.set_title("Year vs Doctors")
        
        elif vis_type == "Pie Chart":
            # Example: pie chart of doctors per country (top 5)
            top_countries = filtered.groupby(group_col)[y_col].sum().nlargest(5)
            ax.pie(top_countries, labels=top_countries.index, autopct='%1.1f%%')
            ax.set_title("Doctors by Top 5 Countries")
        
        else:
            ax.text(0.5, 0.5, f"Unknown visualization: {vis_type}", ha='center')
        
        img = fig_to_pil(fig)
        images.append(img)
    return images


with gr.Blocks() as app:
    
    # State to share filtered data between tabs
    filtered_data = gr.State(df) 

    with gr.Tab("Filter"):
        with gr.Row():
              with gr.Column():
                  country = gr.Dropdown(
                      choices=sorted(df["Countries, territories and areas"].unique().tolist()),
                      value=[],  
                      label="Countries",
                      multiselect=True,
                      allow_custom_value=False,
                      max_choices=50 
                  )
                  year_min = gr.Number(value=df["Year"].min(), label="Min Year")
                  year_max = gr.Number(value=df["Year"].max(), label="Max Year")
                  doctor_min = gr.Number(value=0, label="Min Doctors")
                  gr.Markdown("💡 **Tip:** Hold Ctrl/Cmd to select multiple countries")
                  filter_btn = gr.Button("Apply Filters")
              with gr.Column():
                    filtered_df = gr.Dataframe(headers=list(df.columns), interactive=False)
    
    with gr.Tab("Summary"):
        with gr.Row():
            summary_df = gr.Dataframe(interactive=False, label="Statistical Summary")
        with gr.Row():
            kpi_text = gr.Markdown()
            pie_chart = gr.Image(label="Doctor Types Proportion")

    with gr.Tab("Visualization"):
        with gr.Row():
            with gr.Column():
                visualization_types = gr.Dropdown(
                    choices=[
                          "Heatmap",
                          "Bar Graph",
                          "Line Graph",
                          "Box Plot",
                          "Scatter Plot",
                          "Pie Chart"
                      ],
                      value=["Bar Graph", "Line Graph"], 
                      label="Visualization Types",
                      multiselect=True,
                      max_choices=6, 
                      allow_custom_value=False
                  )

                gr.Markdown("💡 **Tip:** Select the type of graph you want to visualize the data.")
                plot_btn = gr.Button("Generate Visualization")
                gallery = gr.Gallery(label="Visualizations", show_label=True, elem_id="gallery")

    

    
    # Filter tab logic
    plot_btn.click(
        visualization,
        [visualization_types, filtered_data],
        [gallery]
      )

    filter_btn.click(
        filter_dataset,
        [country, year_min, year_max, doctor_min],
        [filtered_df, filtered_data]
        )

    # Auto-update summary when filtered data changes
    filtered_data.change(
        summary,
        filtered_data,
        [summary_df, kpi_text, pie_chart],
        gallery
        )

app.launch(share=True)

* Running on local URL:  http://127.0.0.1:7886
* Running on public URL: https://68fd07d7c7a87f4749.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# For Reference,

- We took help of Youtube videos, Chat GPT, gradio documentation